# Reweighting Tutorial
Some intro here to present the user's desire


In [ ]:
import IO
import os
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from emus import emus, avar, usutils

config_file = '2d_reweighting.config'

In [ ]:
parameters = {}
IO.config_parser(config_file,parameters)